# Utilities

In [25]:
def search_ticker(query):
    stock = yf.Ticker(query)
    info = stock.info
    print(f"Available attributes: {', '.join(info.keys())}")
    
    print(f"Ticker: {query}")
    print(f"Short Name: {info.get('shortName', 'Not Found')}")
    print(f"Long Name: {info.get('longName', 'Not Found')}")
    print(f"Sector: {info.get('sector', 'Not Found')}")
    print(f"Industry: {info.get('industry', 'Not Found')}")
    print(f"Country: {info.get('country', 'Not Found')}")
    print(f"Currency: {info.get('currency', 'Not Found')}")
    print(f"Exchange: {info.get('exchange', 'Not Found')}")
    print(f"Previous Close: {info.get('regularMarketPreviousClose', 'Not Found')}")
    print(f"Market Cap: {info.get('marketCap', 'Not Found')}")
    print("")

In [26]:
# search_ticker("AAPL")
# search_ticker("TSLA")
search_ticker("00631L.TW")  # Yuanta Daily Taiwan 50 Bull 2X ETF

Available attributes: phone, maxAge, priceHint, previousClose, open, dayLow, dayHigh, regularMarketPreviousClose, regularMarketOpen, regularMarketDayLow, regularMarketDayHigh, volume, regularMarketVolume, averageVolume, averageVolume10days, averageDailyVolume10Day, bid, ask, totalAssets, fiftyTwoWeekLow, fiftyTwoWeekHigh, fiftyDayAverage, twoHundredDayAverage, navPrice, currency, ytdReturn, fundFamily, fundInceptionDate, legalType, threeYearAverageReturn, fiveYearAverageReturn, exchange, quoteType, symbol, underlyingSymbol, shortName, longName, firstTradeDateEpochUtc, timeZoneFullName, timeZoneShortName, uuid, messageBoardId, gmtOffSetMilliseconds, trailingPegRatio
Ticker: 00631L.TW
Short Name: YUANTA SECURITIES INV TRUST CO 
Long Name: Yuanta Daily Taiwan 50 Bull 2X ETF
Sector: Not Found
Industry: Not Found
Country: Not Found
Currency: TWD
Exchange: TAI
Previous Close: 207.05
Market Cap: Not Found



# Initialize Spark Session

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg

spark = SparkSession.builder \
    .appName("Backtesting 00631L Strategy") \
    .getOrCreate()

# Load and Prepare data

In [4]:
!pip install yfinance

  Using cached lxml-5.2.2-cp312-cp312-macosx_10_9_universal2.whl.metadata (3.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.9/315.9 kB 1.2 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 6.9 MB/s eta 0:00:000:00:01m eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 3.7 MB/s eta 0:00:00
Using cached lxml-5.2.2-cp312-cp312-macosx_10_9_universal2.whl (8.2 MB)
  Created wheel for frozendict: filename=frozendict-2.4.4-cp312-cp312-macosx_14_0_arm64.whl size=16034 sha256=25d33c455dcc00a211ea73abff6ce9f72a781113cad0bebfaab1c7ce5e48a4b1


In [ ]:
# Load the data into a Spark DataFrame
# Assuming the CSV file contains columns: 'date', 'open', 'high', 'low', 'close', 'volume'
file_path = "path/to/00631L_monthly_data.csv"
data = spark.read.csv(file_path, header=True, inferSchema=True)

# Calculate the 12-month moving average
data = data.withColumn("moving_avg", avg(col("close")).over(Window.orderBy("date").rowsBetween(-11, 0)))

# Show the data with moving average
data.show()

In [6]:
import yfinance as yf
import pandas as pd

def fetch_monthly_data(ticker):
    # Fetch the data
    stock = yf.Ticker(ticker)
    hist = stock.history(period="max")

    # Resample to monthly data
    monthly_data = hist.resample('ME').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    }).dropna()

    return monthly_data

def save_to_csv(data, filename):
    data.to_csv(filename, index=True)

# Fetch and save the data
ticker = "00631L.TW"
monthly_data = fetch_monthly_data(ticker)
save_to_csv(monthly_data, "00631L_monthly_data.csv")
